# train cars price model

Given the cars data that we have in snowflake

* Use xgboost and Scikit kearns random forest
* try MAPIE, for prediction intervals (`!pip install mapie`)

In [1]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark import functions as F
from connection_config import connection_parameters

import pandas as pd

#### Current Environment Details
def current_snowflake_env():
    snowflake_environment = session.sql('select current_user(), current_role(), current_database(), current_schema(), current_version(), current_warehouse()').collect()
    print('User                     : {}'.format(snowflake_environment[0][0]))
    print('Role                     : {}'.format(snowflake_environment[0][1]))
    print('Database                 : {}'.format(snowflake_environment[0][2]))
    print('Schema                   : {}'.format(snowflake_environment[0][3]))
    print('Warehouse                : {}'.format(snowflake_environment[0][5]))
    print('Snowflake version        : {}'.format(snowflake_environment[0][4]))

#### Set up a connection with Snowflake
session = Session.builder.configs(connection_parameters).create()


In [2]:
session

In [3]:
current_snowflake_env()

User                     : SNOWFLAKETRIALUSER
Role                     : ACCOUNTADMIN
Database                 : SNOWFLAKE_SAMPLE_DATA
Schema                   : TPCH_SF10
Warehouse                : COMPUTE_WH
Snowflake version        : 7.34.1


In [10]:
cars_sf = session.table('CARS_DATA.PUBLIC.CARS_CLEAN')

In [11]:
cars = cars_sf.to_pandas()

In [12]:
cars

,BOUWJAAR,KM_STAND,BRANDSTOF,TRANSMISSIE,TYPE,KLEUR,DEUR,MERK,MODEL,VRAAGPRIJS,POWER,MOTOR_INHOUD,AGE,N_DOORS
0,2020,7281,Benzine,Automaat,SUV / Terreinwagen,Rood,5-deurs,CitroÃ«n,C3,26950,96.0,1199.0,3,5
1,2015,26120,Benzine,Handgeschakeld,Hatchback,Wit,3-deurs,Fiat,500,9750,51.0,1242.0,8,3
2,2018,44398,Benzine,Handgeschakeld,SUV / Terreinwagen,Grijs,5-deurs,Ford,EcoSport,15750,73.0,998.0,5,5
3,2014,75633,Benzine,Automaat,MPV,Blauw,5-deurs,Volkswagen,Golf,15199,81.0,1197.0,9,5
4,2017,87990,Benzine,Handgeschakeld,Hatchback,Oranje,5-deurs,Ford,Fiesta,13445,74.0,998.0,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179485,2018,48000,Benzine,Automaat,Hatchback,Grijs,5-deurs,Mercedes-Benz,A-klasse,35250,120.0,1332.0,5,5
179486,2017,62676,Benzine,Automaat,Sedan,Wit,4-deurs,Mercedes-Benz,C-klasse,29900,115.0,1595.0,6,4
179487,2017,76714,Benzine,Automaat,Stationwagon,Zwart,5-deurs,Mercedes-Benz,C-klasse,29690,115.0,1595.0,6,5
179488,2015,103119,Diesel,Automaat,Stationwagon,Grijs,5-deurs,Mercedes-Benz,CLS-klasse,30990,125.0,2143.0,8,5


In [14]:
## create a random forest prediction model with the car data
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# split the data into training and test sets
train, test = train_test_split(cars[["VRAAGPRIJS", "KM_STAND",	"POWER",	"MOTOR_INHOUD", 	"AGE"]], test_size=0.2)

# create a random forest model

rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=0)

# train the model
rf.fit(train.drop('VRAAGPRIJS', axis=1), train['VRAAGPRIJS'])


RandomForestRegressor(max_depth=10, random_state=0)

In [15]:
rf.score(test.drop('VRAAGPRIJS', axis=1), test['VRAAGPRIJS'])

0.9172853878996624